# Backtesting setup
I'm setting up infrastructure to backtest machine learning models. Initially, the models predict the price of certain markets, with the same model trained on all markets. Investment model is very simple based on the predictions. 

In [1]:
import quantiacsToolbox
from quantiacsToolbox import loadData
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model, ensemble, metrics
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline

In [ ]:
## DATA PREP
def fillnans(inArr):
    ''' fills in (column-wise)value gaps with the most recent non-nan value.
    fills in value gaps with the most recent non-nan value.
    Leading nan's remain in place. The gaps are filled in only after the first non-nan entry.
    Args:
        inArr (list, numpy array)
    Returns:
        returns an array of the same size as inArr with the nan-values replaced by the most recent non-nan entry.
    '''
    inArr = inArr.astype(float)
    nanPos = np.where(np.isnan(inArr))
    nanRow = nanPos[0]
    nanCol = nanPos[1]
    myArr = inArr.copy()
    for i in range(len(nanRow)):
        if nanRow[i] > 0:
            myArr[nanRow[i], nanCol[i]] = myArr[nanRow[i] - 1, nanCol[i]]
    return myArr

def mySettings(lookback = 200):
    """ Define your trading system settings here """

    settings = {}

    # Futures Contracts
    settings['markets'] = ['CASH', 'F_AD', 'F_BO', 'F_BP', 'F_C', 'F_CC', 'F_CD',
                           'F_CL', 'F_CT', 'F_DX', 'F_EC', 'F_ED', 'F_ES', 'F_FC', 'F_FV', 'F_GC',
                           'F_HG', 'F_HO', 'F_JY', 'F_KC', 'F_LB', 'F_LC', 'F_LN', 'F_MD', 'F_MP',
                           'F_NG', 'F_NQ', 'F_NR', 'F_O', 'F_OJ', 'F_PA', 'F_PL', 'F_RB', 'F_RU',
                           'F_S', 'F_SB', 'F_SF', 'F_SI', 'F_SM', 'F_TU', 'F_TY', 'F_US', 'F_W', 'F_XX',
                           'F_YM']

    
    settings['budget'] = 10 ** 6
    settings['slippage'] = 0.05

    settings['threshold'] = 0.2
    
    # Train, Val, Test time ranges
    settings['train_start'] = '20100601'
    settings['train_end'] = '20120101'
    settings['val_start'] = '20120101'#settings['train_end']
    settings['val_end'] = '20140101'
#     settings['test_start'] = settings['val_end']
#     settings['test_end'] = '20140101'
    # Lookback is how much historical data to pull for each day
    settings['lookback'] = lookback
    
    return settings

def generate_returns_dict(quant_dict,col='CLOSE'):
    '''Generate daily returns column based on daily price column col, handle NaNs.
    Input is quant_dict, dictionary returned from loadData function.'''
    
    quant_dict['RET'] = generate_returns(quant_dict['CLOSE'],quant_dict['RINFO'])

    return quant_dict

def generate_returns(CLOSE,RINFO):
    RET = np.zeros([1,CLOSE.shape[1]])
    RET = np.append(RET,np.float64(CLOSE[1:, :] - \
                    CLOSE[:-1, :] - RINFO[1:, :]) / CLOSE[:-1, :],axis=0)
    RET = fillnans(RET)
    RET[np.isnan(RET)] = 0
    return RET

def quant_dict_to_df(quant_dict,settings):
    '''Function transforms quant_dict returned from loadData function (plus returns)
    into dataframe. Each row in dataframe is a day for a particular market.'''

    # MARKET and DATE must be in col_list
    if 'markets' not in list(settings.keys()):
        print("MARKET must be in col_list")
        return
    if 'DATE' not in list(quant_dict.keys()):
        print("MARKET must be in col_list")
        return

    # Set up df by market and date
    df = pd.DataFrame([[m,d] for m in settings['markets'] for d in quant_dict['DATE']],columns=['MARKET','DATE'])

    # Flatten out the data and append columns

    for col in quant_dict.keys():
        if col != 'DATE':
            # Fillnans with zero for now
            quant_dict[col][np.isnan(quant_dict[col])] = 0
            df[col] = quant_dict[col].flatten('F')

    return df

In [ ]:
## FEATURE GENERATION


## MODEL EVALUATION
def eval_model(reg, X_train, X_val, y_train, y_val, 
               metric_dict, binary_list, metric_df = None):
    '''eval_model() generates train and val evaluation metrics for reg
    regression model.
    Inputs:
    reg - Pre-trained regression model 
    X_train, X_val - Feature Dataframes
    y_train, y_val - Returns to predict
    metric_dict - Dictionary of metrics to calculate
    binary_list - List of metrics that operate on sign(y)
    metrif_df - optional, if provided then metrics for reg will be appended
    
    Outputs:
    metric_df - DataFrame w/ rows of metric_list and columns train and val
    '''
    # Predictions
    y_train_pred = reg.predict(X_train)
    y_val_pred = reg.predict(X_val)

    # Define the empty metrics dataframe (if not provided)
    if metric_df is None:
        df_ix = [a+b for b in ['_train','_val'] for a in metric_dict]
        metric_df = pd.DataFrame(index=df_ix)
    
    # Metrics calculation loop
    reg_name = str(reg)[:15]
    for met in metric_dict:
        if met in binary_list:
            metric_df.loc[met+'_train',reg_name] = metric_dict[met]( \
                          np.sign(y_train),np.sign(y_train_pred))
            metric_df.loc[met+'_val',reg_name] = metric_dict[met]( \
                          np.sign(y_val),np.sign(y_val_pred))
        else:
            metric_df.loc[met+'_train',reg_name] = metric_dict[met]( \
                          y_train,y_train_pred)
            metric_df.loc[met+'_val',reg_name] = metric_dict[met]( \
                          y_val,y_val_pred)
    return metric_df

## Data Prep

In [ ]:
# Load Settings
settings = mySettings()
dataToLoad = set(['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'P', 'RINFO'])

# Train, Val, and Test
train_dict = loadData(marketList=settings['markets'], dataToLoad=dataToLoad, refresh=False, 
                    beginInSample=settings['train_start'], endInSample=settings['train_end'],
             dataDir="../tickerData")
val_dict = loadData(marketList=settings['markets'], dataToLoad=dataToLoad, refresh=False, 
                    beginInSample=settings['val_start'], endInSample=settings['val_end'],
             dataDir="../tickerData")
# test_dict = loadData(marketList=settings['markets'], dataToLoad=dataToLoad, refresh=False, 
#                     beginInSample=settings['test_start'], endInSample=settings['test_end'],
#              dataDir="../tickerData")

train_dict = generate_returns_dict(train_dict)
# train_df = quant_dict_to_df(train_dict,settings)

val_dict = generate_returns_dict(val_dict)
# val_df = quant_dict_to_df(val_dict,settings)

In [ ]:
# LAG
def lag(RETURNS,lag):
    '''lag returns a pd.Series of the lagged values in col, and the df with the lag column appended. 
       First #(lag) rows are zeroes (may change this later to remove these columns)'''

    out = np.zeros([lag,RETURNS.shape[1]])
    out = np.append(out, RETURNS[:-lag,:],axis=0)
    return out

def lag_feats(RETURNS, lag_list, remove_rows=True):
    '''lag_feats generates lag features, as defined by lag_list, 
    and removes initial rows that do not have previous information for lag.
    Output is a list of arrays with RETURNS.shape.'''
    # Add lag columns
    out_list = []
    for l in lag_list:
        out_list.append(lag(RETURNS,l))
    if remove_rows:
        # Remove the first max_lag rows from each list
        max_lag = np.max(lag_list)
        for i,ll in enumerate(out_list):
            out_list[i] = ll[max_lag:]            
    return out_list

## Feature Generation

In [ ]:
# Generate returns
# lag_list = [1,2,3,4,5,10,20,50]
lag_list = [1,2,3,5,10,25,50,100,150,200]
train_dict['LAG_LIST'] = lag_feats(train_dict['RET'], lag_list, False)
val_dict['LAG_LIST'] = lag_feats(val_dict['RET'], lag_list, False)

num_feats = len(lag_list)

In [ ]:
def shape_lag_features(data_dict,num_feats=num_feats):
    '''This function reshapes data_dict into X and y, accounting for lags.
    Should eventually be expanded to hangle other features.'''
    
    max_lag = np.max(lag_list)

    # Create X and y
    y = data_dict['RET'][max_lag:,:].flatten('F')
    full_lag_arr = np.array(data_dict['LAG_LIST'])
    print("full lag arr shape: ",full_lag_arr.shape)
    # Remove early returns that don't have lag
    lag_arr = full_lag_arr[:,max_lag:,:]

    # Reshape for model
    lag_arr = lag_arr.reshape((lag_arr.shape[0],-1),order='F').T
    y = y.reshape(-1,order='F')

    return lag_arr, y

In [ ]:
X_train, y_train = shape_lag_features(train_dict)
X_val, y_val = shape_lag_features(val_dict)

## Train Models

In [ ]:
# Define and fit the model
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

# Dictionary of metrics to calculate
metric_dict = {"r2": metrics.r2_score,
               "MSE": metrics.mean_squared_error,
               "Accuracy": metrics.accuracy_score}
# List of metrics that operate on sign(y_pred)
binary_list = ['Accuracy']

metric_df = eval_model(reg, X_train, X_val, y_train, y_val, 
               metric_dict, binary_list)

In [ ]:
reg2 = ensemble.RandomForestRegressor()
reg2.fit(X_train,y_train)

In [ ]:
metric_df = eval_model(reg2, X_train, X_val, y_train, y_val, 
               metric_dict, binary_list, metric_df)

In [ ]:
from tabulate import tabulate
pdtabulate=lambda df:tabulate(df,headers='keys',tablefmt='psql')
print(pdtabulate(metric_df))

In [ ]:
plt.figure(figsize=[20,20])
plt.plot(y_val)
y_val_pred = reg.predict(X_val)
plt.plot(y_val_pred)

## Trading strategy

In [ ]:
class myStrategy(object):

    def myTradingSystem(self, DATE, OPEN, HIGH, LOW, CLOSE, VOL, OI, P, R, RINFO, exposure, equity, settings):
        
        # Get parameters from setting
        nMarkets = len(settings['markets'])
        lookback = settings['lookback']
        threshold = settings['threshold']

#         pos = np.zeros(nMarkets, dtype=np.float)
        
        # Generate features for a single day
        # Returns
        RETURNS = generate_returns(CLOSE,RINFO)
        # Lags
        lag_list = [1,2,3,5,10,25,50,100,150,200]
        LAG_LIST = lag_feats(RETURNS, lag_list, False)
        # Keep only the last day
        X = np.array(LAG_LIST)[:,-1,:].T

        reg = settings['model']
        try:
            trend = reg.predict(X)
            # Threshold
            trend = (abs(trend) > threshold)*trend

            pos = np.sign(trend)

        # for NaN data set position to 0
        except ValueError:
            print("trend isnan", np.isnan(trend).any())
            print("pos isnan", np.isnan(trend).any())
            pos[market] = .0

        return pos, settings
    
    def mySettings(self):
        """ Define your trading system settings here """

        settings = {}

        # Futures Contracts
        settings['markets'] = ['CASH', 'F_AD', 'F_BO', 'F_BP', 'F_C', 'F_CC', 'F_CD',
                               'F_CL', 'F_CT', 'F_DX', 'F_EC', 'F_ED', 'F_ES', 'F_FC', 'F_FV', 'F_GC',
                               'F_HG', 'F_HO', 'F_JY', 'F_KC', 'F_LB', 'F_LC', 'F_LN', 'F_MD', 'F_MP',
                               'F_NG', 'F_NQ', 'F_NR', 'F_O', 'F_OJ', 'F_PA', 'F_PL', 'F_RB', 'F_RU',
                               'F_S', 'F_SB', 'F_SF', 'F_SI', 'F_SM', 'F_TU', 'F_TY', 'F_US', 'F_W', 'F_XX',
                               'F_YM']

        settings['budget'] = 10 ** 6
        settings['slippage'] = 0.05

        settings['threshold'] = 1e-5

        # Train, Val, Test time ranges
        settings['train_start'] = '20100601'
        settings['train_end'] = '20120101'
        settings['val_start'] = '20120101'#settings['train_end']
        settings['val_end'] = '20140101'

        settings['beginInSample'] = settings['val_end']
        settings['endInSample'] = '20160101'
        # Lookback is how much historical data to pull for each day
        settings['lookback'] = 200
        
        settings['model'] = reg2
        
        return settings

In [ ]:
with np.errstate(invalid='ignore', divide='ignore'):
    result = quantiacsToolbox.runts(myStrategy)

In [ ]:
np.isnan(np.array([1,2,3,np.NaN])).any()